# Merge ALL freqs

In [1]:
import pandas as pd

In [2]:
all_freqs_p = "freqs/all/all-freqs-sketchengine.csv"
andrea_freqs = "freqs/all/all-freqs-sketchengine-andrea.csv"
giulia_freqs = "freqs/all/all-freqs-sketchengine-giulia.csv"
giulia2_freqs = "freqs/all/all-freqs-sketchengine-giulia2.csv"
cate_freqs = "freqs/all/all-freqs-sketchengine-caterina.csv"

In [3]:
all_df = pd.read_csv(all_freqs_p)

# save a backup copy of original freqs
all_df.to_csv(f"{all_freqs_p}.bak", index=False)

andrea_df = pd.read_csv(andrea_freqs)
giulia_df = pd.read_csv(giulia_freqs)
giulia2_df = pd.read_csv(giulia2_freqs)
cate_df = pd.read_csv(cate_freqs)


new_df = pd.concat([all_df, andrea_df, giulia_df, cate_df, giulia2_df])
new_df = new_df.drop_duplicates(subset="exemplar")  # drop duplicated exemplars

new_df.to_csv(all_freqs_p, index=False)
